In [198]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

import time
import string
from collections import defaultdict, deque
from sklearn.model_selection import StratifiedKFold


In [199]:
os.getcwd() 

'C:\\Users\\Lenovo\\Desktop\\stupidcode\\kaggle\\jigsaw'

In [200]:
input_dir=r"C:\Users\Lenovo\Desktop\stupidcode\data\kaggle\jigsaw"

In [201]:
data_df=pd.read_csv(os.path.join(input_dir,"validation_data.csv"))

In [202]:
data_df.head()

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"


In [203]:
from sklearn.preprocessing import LabelEncoder
def generate_comments(data):
    more_toxic_text=data["more_toxic"].values
    less_toxic_text=data["less_toxic"].values    
    comments=np.concatenate((more_toxic_text,less_toxic_text))
    comments=np.unique(comments)
    comments=pd.DataFrame({"text":comments})
    text_encoder=LabelEncoder()
    text_encoder.fit(comments)
    comments["encode_text"]=text_encoder.transform(comments["text"])
    data["encode_less"]=text_encoder.transform(data["less_toxic"])
    data["encode_more"]=text_encoder.transform(data["more_toxic"])
    
    return data,comments

In [204]:
data_df,comments=generate_comments(data_df)

D:\anaconda\envs\python36\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [205]:
data_df.head()

,worker,less_toxic,more_toxic,encode_less,encode_more
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...,2405,12151
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...,7215,653
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu...",2632,7222
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...,7973,12968
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist",3524,3266


In [206]:
data_df.duplicated(subset=["encode_less","encode_more"],keep="first")
data_drop=data_df.drop_duplicates(["encode_less","encode_more"],keep="first",inplace=False)

In [207]:
data_df["label_min"]=data_df.apply(lambda row:row["encode_less"] 
                                   if row["encode_less"]<row["encode_more"] else row["encode_more"],axis=1)
data_df["label_max"]=data_df.apply(lambda row:row["encode_more"] 
                                   if row["encode_less"]<row["encode_more"] else row["encode_less"],axis=1)

data_df["win_min"]=data_df.apply(lambda row:1 if row["encode_more"]<row["encode_less"] else 0 ,axis=1)
data_df["win_max"]=data_df.apply(lambda row:0 if row["encode_more"]<row["encode_less"] else 1 ,axis=1)

data_df_agg=data_df.groupby(["label_min","label_max"]).agg({"win_min":"sum","win_max":"sum"}).reset_index()
data_df_agg["encode_less"]=data_df_agg.apply(lambda row:row["label_min"] 
                                             if row["win_min"]<row["win_max"] else row["label_max"],axis=1)
data_df_agg["encode_more"]=data_df_agg.apply(lambda row:row["label_min"] 
                                             if row["win_min"]>row["win_max"] else row["label_max"],axis=1)

In [210]:
data_df_agg.head()

,label_min,label_max,win_min,win_max,encode_less,encode_more
0,0,7085,2,1,7085,0
1,1,1293,3,0,1293,1
2,1,5156,3,0,5156,1
3,1,12696,2,1,12696,1
4,2,13776,2,1,13776,2


In [21]:
more2less_dict= defaultdict(list)
data_df_agg.apply(lambda row:more2less_dict[row["encode_more"]].append(row["encode_less"]),axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
10103    None
10104    None
10105    None
10106    None
10107    None
Length: 10108, dtype: object

In [30]:
def bsearch(start,more2less_dict):
    queue = deque([start])
    visit_list=[]
    while len(queue)!=0:
        visit_id=queue.popleft()
        if visit_id in visit_list:
            continue
        visit_list.append(visit_id)
        queue+=deque(more2less_dict[visit_id])
    visit_list.remove(start)
    return [ x for x in visit_list if x not in more2less_dict[start] ]

def search_lessText(more2less_dict=more2less_dict):
    aug_dict= defaultdict(list)
    for start in list(more2less_dict.keys()):
        
        aug_list=bsearch(start,more2less_dict)
        aug_dict[start]=aug_list
    return aug_dict
        

In [124]:
aug_dict=search_lessText(more2less_dict)
aug_dict={key:value for key,value in aug_dict.items() if len(value)!=0}
aug_df=pd.DataFrame(columns=(tuple(data_df.columns)))
id2text_dict=comments.to_dict()["text"]

In [172]:
for key,value in aug_dict.items():
    encode_more=key
    encode_less_list=value

    more_toxic=id2text_dict[encode_more]
    for encode_less in encode_less_list:
        less_toxic=id2text_dict[encode_less]
        row=pd.DataFrame({"worker":[999],"less_toxic":[less_toxic],"more_toxic":[more_toxic],"encode_less":[encode_less],
                                   "encode_more":[encode_more]})
        aug_df=aug_df.append(row,ignore_index=True)

work_list=np.array([999]*len(aug_df),dtype=np.int64)
aug_df["worker"]=work_list
data_df_aug=pd.concat([data_df,aug_df],axis=0)
data_df_aug=data_df_aug.reset_index(drop=True)

In [197]:
skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=2021)
for fold,(_,val_) in enumerate(skf.split(X=data_df_aug,y=data_df_aug.worker)):
    data_df_aug.loc[val_,"kfold"]=int(fold)
    
data_df_aug["kfold"]=data_df_aug["kfold"].astype(int)
data_df_aug.head()

D:\anaconda\envs\python36\lib\site-packages\sklearn\model_selection\_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


,worker,less_toxic,more_toxic,encode_less,encode_more,label_min,label_max,win_min,win_max,kfold
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...,2405,12151,2405,12151,0,1,4
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...,7215,653,653,7215,1,0,0
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu...",2632,7222,2632,7222,0,1,0
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...,7973,12968,7973,12968,0,1,2
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist",3524,3266,3266,3524,1,0,0
